In [1]:
!pip install selenium
!pip install chromedriver-autoinstaller
!pip install pandas
!pip install requests
!pip install bs4

In [2]:
import pandas as pd
import time
import requests
import chromedriver_autoinstaller

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

In [3]:
def scroll_to_bottom(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [4]:
def click_element(driver, xpath):
    driver.find_element(By.XPATH, xpath).click()

In [5]:
def scrape_page(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tabela = soup.find('div', {'class': {'col-lg-11 col-md-11 col-sm-12'}})
    titulos = tabela.find_all('strong', {})
    conteudos = tabela.find_all('p')[0::2]
    datas_publicacao = tabela.find_all('p', {'class': 'data-hora'})

    lista_titulos = [titulo.get_text().strip() for titulo in titulos]
    lista_conteudos = [conteudo.get_text().strip() for conteudo in conteudos]
    lista_datas_publicacao = [data.get_text().strip() for data in datas_publicacao]

    return pd.DataFrame([lista_titulos, lista_conteudos, lista_datas_publicacao]).T

In [6]:
service = webdriver.chrome.service.Service()

options = webdriver.ChromeOptions()

driver = webdriver.Chrome(service=service, options=options)

In [7]:
driver.get("https://anmlegis.datalegis.net/action/ActionDatalegis.php?acao=apresentacao&cod_modulo=351&cod_menu=8014")

In [8]:
# Click em "Títulos Outorgados"
click_element(driver, '//*[@id="column-2"]/div[1]/a[4]')

In [9]:
# Click em "Permissão de Lavra Garimpeira - PLG"
click_element(driver, '//*[@id="column-2"]/div/div[2]/div[4]/ul/li[5]/a')

In [10]:
# Click em "2024"
click_element(driver, '//*[@id="column-2"]/div[1]/ul/li[1]/a')
scroll_to_bottom(driver)
df_plg2024 = scrape_page(driver)
driver.back()

In [11]:
# Click em "2023"
click_element(driver, '//*[@id="column-2"]/div[1]/ul/li[2]/a')
scroll_to_bottom(driver)
df_plg2023 = scrape_page(driver)
driver.back()

In [12]:
# Click em "2022"
click_element(driver, '//*[@id="column-2"]/div[1]/ul/li[3]/a')
scroll_to_bottom(driver)
df_plg2022 = scrape_page(driver)
driver.back()

In [13]:
# Click em "2021"
click_element(driver, '//*[@id="column-2"]/div[1]/ul[1]/li[4]/a')
scroll_to_bottom(driver)
df_plg2021 = scrape_page(driver)
driver.back()

In [14]:
# Click em "2020"
click_element(driver, '//*[@id="column-2"]/div[1]/ul/li[5]/a')
scroll_to_bottom(driver)
df_plg2020 = scrape_page(driver)
driver.back()

In [15]:
# Click em "2019"
click_element(driver, '//*[@id="column-2"]/div[1]/ul/li[6]/a')
scroll_to_bottom(driver)
df_plg2019 = scrape_page(driver)
driver.back()

In [16]:
# Click em "2018"
click_element(driver, '//*[@id="column-2"]/div[1]/ul/li[7]/a')
scroll_to_bottom(driver)
df_plg2018 = scrape_page(driver)
driver.back()

In [17]:
plg = pd.concat([df_plg2018, df_plg2019, df_plg2020, df_plg2021, df_plg2022, 
                df_plg2023, df_plg2024])

In [18]:
plg

,0,1,2
0,LICENCIAMENTO Nº 35/2018,"Outorga o Licenciamento, em nome de JOSE DAMIÃ...",11/05/2018 | 07:25:20
1,LICENCIAMENTO Nº 001/2018,"Outorga o Licenciamento, em nome de AREIA DA L...",01/02/2018 | 12:28:49
0,PERMISSÃO DE LAVRA GARIMPEIRA Nº 187/2019 DE 5...,"Outorga pelo prazo de 5 ano(s), COOPERATIVA DE...",09/12/2019 | 10:39:27
1,PERMISSÃO DE LAVRA GARIMPEIRA Nº 186/2019 DE 2...,"Outorga pelo prazo de 5 ano(s), COOPERATIVA DE...",04/12/2019 | 10:36:45
2,PERMISSÃO DE LAVRA GARIMPEIRA Nº 185/2019 DE 2...,"Outorga pelo prazo de 5 ano(s), RIBEIRO OMAR D...",04/12/2019 | 10:34:15
...,...,...,...
40,"PERMISSÃO DE LAVRA GARIMPEIRA N° 6/2024, de 23...","Outorga a JOÃO EVANGELISTA ALVES, pelo prazo d...",25/01/2024 | 11:31:45
41,"PERMISSÃO DE LAVRA GARIMPEIRA N° 5/2024, de 23...","Outorga a JOÃO EVANGELISTA ALVES, pelo prazo d...",25/01/2024 | 11:29:38
42,PERMISSÃO DE LAVRA GARIMPEIRA Nº 1/2024,"Outorga até POR 05 (CINCO) ANOS, a Robson Lana...",19/01/2024 | 08:59:28
43,PERMISSÃO DE LAVRA GARIMPEIRA Nº 4/2024,"Outorga POR 05 (CINCO) ANOS, a Sandro Andrade ...",17/01/2024 | 10:07:59
